### Attention Mechanism

앞서 나온 seq2seq 모델은 인코더에서 입력 시퀀스를 컨텍스트 벡터라는 하나의 고정된 크기의 벡터 표현으로 압축하고, 디코더는 이 컨텍스트 벡터를 통해서 출력 시퀀스를 만들어냈습니다.

But. 문제가 있습니다.
(1) 하나의 고정된 크기의 벡터에 모든 정보를 압축하려고 하니까 정보 손실이 발생합니다.
(2) RNN의 고질적인 문제인 기울기 소실(vanishing gradient)문제가 존재합니다.

이를 위한 대안으로 입력 시퀀스가 길어지면 출력 시퀀스의 정확도가 떨어지는 것을 보정해주기 위해 등장한 기법인 Attention을 소개합니다.

#### 1. Attention의 아이디어

어텐션의 기본 아이디어는 디코더에서 출력 단어를 예측하는 매 시점(time step)마다, 인코더에서의 전체 입력 문장을 다시 한번 참고한다는 점입니다. 단, 전체 입력 문장을 전부 다 동일한 비율로 참고하는 것이 아니라, 해당 시점에서 예측해야할 단어와 관련이 있는 입력 단어 부분을 좀 더 집중(attention)해서 보게 됩니다.

#### 2. Attention Function


파이썬의 딕셔너리 자료형은 키와 값이라는 두 개의 쌍으로 구성되는데, 키를 통해서 맵핑된 값을 찾아낼 수 있다는 특징을 갖고있습니다.

![Alt text](image-30.png)

어텐션 함수는 주어진 쿼리(Query)에 대해서 모든 키(Key)와의 유사도를 각각 구합니다. 그리고 구해낸 이 유사도를 키와 맵핑되어있는 각각의 값(Value)에 반영해줍니다. 그리고 유사도가 반영된 값을 모두 더해서 리턴합니다. 여기서는 이를 어텐션 값(Attention Value)이라고 하겠습니다.

seq2seq + 어텐션 모델에서 Q, K, V에 해당되는 각각의 Query, Keys, Values는 각각 다음과 같습니다.

Q = Query : t 시점의 디코더 셀에서의 은닉 상태

K = Keys : 모든 시점의 인코더 셀의 은닉 상태들

V = Values : 모든 시점의 인코더 셀의 은닉 상태들

#### 3. 닷-프로덕트 어텐션(Dot-Product Attention)

어텐션은 다양한 종류가 있는데 그 중에서도 가장 수식적으로 이해하기 쉽게 수식을 적용한 Dot-Procut Attention을 통해 어텐션을 이해해봅시다. seq2seq에서 사용되는 어텐션 중에서 닷-프로덕트 어텐션의 차이는 주로 중간 수식의 차이로 메커니즘 자체는 거의 유사합니다.

![Alt text](image-31.png)

위 그림은 디코더의 세번째 LSTM 셀에서 출력 단어를 예측할 때, 어텐션 메커니즘을 사용하는 모습을 보여줍니다. 디코더의 첫번째, 두번째 LSTM 셀은 이미 어텐션 메커니즘을 통해 je와 suis를 예측하는 과정을 거쳤다고 가정합니다.

1) 어텐션 스코어(Attention Score)를 구한다.
인코더의 시점(time step)을 각각 1,2,...N이라고 하였을 때 인코더의 은닉 상태(hidden state)를 각각 h1, h2, ..., hN라고 합시다. 디코더의 현재 시점 t에서의 디코더의 은닉 상태를 St라고 합시다. 또한 여기서는 인코더의 은닉 상태와 디코더의 은닉 상태의 차원이 같다고 가정합니다.

시점 t에서 출력 단어를 예측하기 위해서 디코더의 셀은 두 개의 입력값을 필요로 하는데, 바로 이전 시점인 t-1의 은닉 상태와 이전 시점 t-1에 나온 출력 단어입니다.

그런데 어텐션 메커니즘에서는 출력 단어 예측에 또 다른 값을 필요로 하는데 바로 어텐션 값(Attention Value)이라는 새로운 값입니다. t번째 단어를 예측하기 위한 어텐션 값을 at이라고 정의하겠습니다.

텐션 스코어란 현재 디코더의 시점 t에서 단어를 예측하기 위해, 인코더의 모든 은닉 상태 각각이 디코더의 현 시점의 은닉 상태 St와 얼마나 유사한지를 판단하는 스코어값입니다.

![Alt text](image-32.png)

2) 소프트맥스(softmax) 함수를 통해 어텐션 분포(Attention Distribution)를 구한다. 

![Alt text](image-33.png)

3) 각 인코더의 어텐션 가중치와 은닉 상태를 가중합하여 어텐션 값(Attention Value)를 구한다.어텐션의 최종 결과값을 얻기 위해서 각 인코더의 은닉 상태와 어텐션 가중치값들을 곱하고, 최종적으로 모두 더합니다. 

![Alt text](image-34.png)

4) 어텐션 값과 디코더의 t 시점의 은닉 상태를 연결한다.(Concatenate)

![Alt text](image-35.png)

5) 출력층 연산의 입력이 되는 St를 계산합니다.

![Alt text](image-36.png)

6) St를 출력층의 입력으로 사용합니다.

![Alt text](image-37.png)


#### 4. 다양한 종류의 어텐션(Attention)

![Alt text](image-38.png)